In [187]:
from langchain_core.tools import tool
import random
import requests
from langchain_openai import ChatOpenAI
import os
from langgraph.prebuilt import create_react_agent

### tool

In [190]:
@tool
def random_movie(genre: str) -> str: #explicit type hint 
    """Returns a random movie name based on the given genre."""
    url = "https://api.npoint.io/1b111baad7e6ea24a1f6"

    genre=genre.title()
    response = requests.get(url).json()
    
    if genre not in response:
        return f"No movies found for genre: {genre}"
    
    movie = random.choice(response[genre])
    return movie
random_movie('comedy')

'Superbad'

### Model

In [193]:


from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=os.environ["TOGETHERAI_API_KEY"],
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
)

In [194]:
#bind_tools to give model knowledge about these tools
tools = [random_movie]
model_with_tools = model.bind_tools(tools)

In [197]:
tools

[StructuredTool(name='random_movie', description='Returns a random movie name based on the given genre.', args_schema=<class 'langchain_core.utils.pydantic.random_movie'>, func=<function random_movie at 0x0000024F65F52B60>)]

In [199]:
model_with_tools.invoke('hi').tool_calls

[]

In [200]:
model_with_tools.invoke('any Comedy movie').tool_calls

[{'name': 'random_movie',
  'args': {'genre': 'Comedy'},
  'id': 'call_w0t59xhculsrrvnswe1e1kir',
  'type': 'tool_call'}]

### model with tool

In [205]:
prompt = '''Act as a helpful assistant.

        - random_movie - only use the tool when user asks for movie recommendation.
            
        check the genre used by user as Comedy , Action , Drama, Horror,Scifi and recommed the user with the movie

        use the tool only when you do not know the answer
        


'''

print(model_with_tools.invoke(prompt).content)
print(model_with_tools.invoke(prompt).tool_calls)

I'm ready to help. What kind of movie are you in the mood for? Are you looking for a Comedy, Action, Drama, Horror, or Scifi movie? Let me know and I'll do my best to recommend one.
[]


In [206]:

agent = create_react_agent(model=model, tools=tools, state_modifier=prompt)

In [207]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]

        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print() 

inputs = {"messages": [("user", "hi")]}

print_stream(agent.stream(inputs, stream_mode="values"))  # Ensure agent is properly defined


================================ Human Message =================================

hi
================================== Ai Message ==================================

It's nice to meet you. Is there something I can help you with or would you like to chat? Maybe you're looking for a movie recommendation?


In [185]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        #print(message)

        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print() 

inputs = {"messages": [("user", "Comedy movie recommend")]}

print_stream(agent.stream(inputs, stream_mode="values"))  # Ensure agent is properly defined


================================ Human Message =================================

Comedy movie recommend
================================== Ai Message ==================================
Tool Calls:
  random_movie (call_nsdt5d31d8qvj4hylknxjdwq)
 Call ID: call_nsdt5d31d8qvj4hylknxjdwq
  Args:
    genre: Comedy
================================= Tool Message =================================
Name: random_movie

The Grand Budapest Hotel
================================== Ai Message ==================================

I hope you enjoy "The Grand Budapest Hotel"! If you're looking for more comedy movie recommendations, please let me know!
